In [1]:
from langchain_community.document_loaders import WebBaseLoader, SeleniumURLLoader

In [2]:
urls = [
    'https://finance.yahoo.com/news/better-ai-stock-nvidia-vs-183000082.html'
]

In [4]:
web_base_loader = WebBaseLoader(urls)
selenium_loader = SeleniumURLLoader(urls = urls)

In [5]:
loaded_url = web_base_loader.load()

In [6]:
selenium_url_loader = selenium_loader.load()

In [7]:
loaded_url

[Document(page_content="Better AI Stock: Nvidia vs. Super Micro Computer        HOME    MAIL    NEWS    FINANCE    SPORTS    ENTERTAINMENT    LIFE    SEARCH    SHOPPING    YAHOO PLUS    MORE...                  Yahoo Finance      Sign in      Mail  Sign in to view your mail      Finance     Watchlists     My Portfolio    Markets    YF Chartbook Calendars Trending Tickers Stocks: Most Actives Stocks: Gainers Stocks: Losers Top ETFs Futures World Indices Currencies Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility US Treasury Bonds Rates Currency Converter    News    Latest News From the Newsroom Stock Market News Earnings Politics Economic News Morning Brief Personal Finance News Crypto News Bidenomics Report Card    Videos    Yahoo Finance Invest Yahoo Finance Live ETF Report Options 101 Good Buy or Goodbye FA Corner     Yahoo Finance Plus    Dashboard Research Reports Investment Ideas Community Insights Webinars Blog    Screeners    Saved Screeners Eq

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=200)

In [11]:
chunks = text_splitter.split_documents(loaded_url)

In [12]:
len(chunks)

10

In [13]:
chunks[1]

Document(page_content='Options 101 Good Buy or Goodbye FA Corner     Yahoo Finance Plus    Dashboard Research Reports Investment Ideas Community Insights Webinars Blog    Screeners    Saved Screeners Equity Screener Mutual Fund Screener ETF Screener Futures Screener Index Screener Analyst Rating Screener   Technical Events Screener   Smart Money Screener   Top Holdings Screener      Personal Finance    Credit Cards Banking Personal loans Student loans Car insurance Home Buying Taxes     Crypto    Sectors    Basic Materials Communication Services Consumer Cyclical Consumer Defensive Energy Financial Services Healthcare Industrials Real Estate Technology Utilities     Contact Us    …   AdvertisementU.S. markets closedS&P 5005,117.94-5.75 (-0.11%)\xa0Dow 3038,769.66+46.97 (+0.12%)\xa0Nasdaq16,019.27-65.84 (-0.41%)\xa0Russell 20002,065.88-16.83 (-0.81%)\xa0Crude Oil78.14+0.13 (+0.17%)\xa0Gold2,189.00+3.50 (+0.16%)\xa0Silver24.68+0.13 (+0.53%)\xa0EUR/USD1.0929-0.0013 (-0.12%)\xa010-Yr Bond4

In [14]:
from langchain_google_vertexai import VertexAIEmbeddings
from dotenv import load_dotenv
import os
import google.generativeai as genai

google_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_api_key)
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@003")

In [15]:
query = "How much did AMD grew last quarter?"

In [17]:
query_embedded = embeddings.embed_query(query)

In [18]:
len(query_embedded)

768

In [19]:
from langchain_community.vectorstores import FAISS

In [22]:
vector_store = FAISS.from_documents(documents = chunks, embedding = embeddings)

In [23]:
vector_store.save_local("faiss_index")

In [24]:
from langchain_google_vertexai import VertexAI

In [25]:
llm = VertexAI(model_name="text-bison@002")

In [26]:
from langchain.prompts import PromptTemplate    
prompt_template = """
Human: Answer the question as detailed as possible from the provided context, make sure to provide all the details. 
    Don't exceed 250 words on the explanation. If you don't know the answer, just say that you don't know, 
    don't try to make up an answer.\n
    
    Context:\n {context}?\n
    Question: {question}
    
    Assistant:
"""
prompt = PromptTemplate(template = prompt_template, input_variables = ['context', 'question'])

In [28]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_store.as_retriever(
        search_type = "similarity", search_kwargs = {"k": 3}
    ),
    chain_type = "stuff",
    return_source_documents = True,
    chain_type_kwargs={"prompt": prompt}
    
)

In [30]:
question = "What is the stock price increase of Nvidia in 2023?"

In [31]:
answer = qa({"query": question})

In [32]:
answer

{'query': 'What is the stock price increase of Nvidia in 2023?',
 'result': " Nvidia's stock has experienced significant growth in 2023. Since the beginning of the year, its stock price has increased by an impressive 530%. This remarkable performance has made Nvidia a very attractive investment opportunity for many investors. However, it's important to note that past performance does not guarantee future results, and the stock market is always subject to fluctuations and risks.",
 'source_documents': [Document(page_content="(ending Jan. 28), Nvidia's revenue rose 265% year over year to $22.1 billion. It also gave guidance for Q1 revenue of $24 billion, indicating 234% growth.Supermicro also delivered strong results, with its Q2 FY 2024 (ending Dec. 31) revenue rising 103% to $3.67 billion. For its next quarter, Supermicro projects growth of 188% to 219%, placing it in the same realm as Nvidia.While Nvidia experienced the first wave of GPU demand, Supermicro's growth is just starting. H